## Assignment 11.2
Author: Rex Gayas
Date: 25 February 2024
Modified By: N/A
Description: Interfacing with SQL databases through Python to perform tasks such as creating tables, inserting and querying data, as well as utilizing ORM tools like SQLAlchemy for database interactions and Pandas for data manipulation.

In [13]:
# Solutions to Task 1
import sqlite3
import random

# Connect to the SQLite database. Using ':memory:' to create a temporary database in RAM.
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create the sensors table
cursor.execute('''
CREATE TABLE sensors (
    id INTEGER PRIMARY KEY,
    type TEXT,
    location TEXT,
    reading REAL
)''')
print('Table "sensors" has been created.')

# Confirm the table creation by querying the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print('Existing tables:', cursor.fetchall())

# Insert random data into the sensors table
for _ in range(5):
    cursor.execute('''
    INSERT INTO sensors (type, location, reading) VALUES (?, ?, ?)
    ''', (random.choice(['Temperature', 'Humidity', 'Pressure']),
          random.choice(['Room 1', 'Room 2', 'Room 3']),
          random.uniform(20.0, 30.0)))

# Commit the insertions
conn.commit()

# Select all results from the sensors table
cursor.execute('SELECT * FROM sensors')
print('All data from "sensors" table:', cursor.fetchall())

# Drop the sensors table
cursor.execute('DROP TABLE sensors')
print('Table "sensors" has been dropped.')

# Try to confirm the table has been dropped by selecting from it
try:
    cursor.execute('SELECT * FROM sensors')
except sqlite3.OperationalError as e:
    print('Error:', e)

# Close the connection
conn.close()
print('Connection closed.')


Table "sensors" has been created.
Existing tables: [('sensors',)]
All data from "sensors" table: [(1, 'Humidity', 'Room 1', 22.896731865119563), (2, 'Humidity', 'Room 3', 23.011791223958923), (3, 'Humidity', 'Room 3', 25.279644155443247), (4, 'Temperature', 'Room 1', 21.00348784677767), (5, 'Humidity', 'Room 1', 29.914798220867066)]
Table "sensors" has been dropped.
Error: no such table: sensors
Connection closed.


In [14]:
# Solutions to Task 2

import sqlite3
import pandas as pd

# Create a list of tuples to form the pandas DataFrame
data_tuples = [
    (1700, 5),
    (1701, 11),
    (1702, 16),
    (1703, 18),
    (1704, 11),
    (1705, 22),
    (1731, 11)
]

# Convert list of tuples into a pandas DataFrame
df = pd.DataFrame(data_tuples, columns=['year', 'sunactivity'])

# Connect to the SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a table without specifying data types
cursor.execute('''
CREATE TABLE sunspots (
    year,
    sunactivity
)''')
print('Table "sunspots" has been created without specifying data types.')

# Insert records from the list of tuples into the table
cursor.executemany('INSERT INTO sunspots (year, sunactivity) VALUES (?, ?)', data_tuples)
conn.commit()
print('Inserted', cursor.rowcount, 'records into the table.')

# Delete records where sunactivity is greater than 20
cursor.execute('DELETE FROM sunspots WHERE sunactivity > 20')
conn.commit()
print('Deleted', cursor.rowcount, 'records from the table where sunactivity is greater than 20.')

# Write a query to select all data where the year is less than 1732
cursor.execute('SELECT * FROM sunspots WHERE year < 1732')
rows = cursor.fetchall()
print('Data where the year is less than 1732:')
for row in rows:
    print(row)

# Close the connection
conn.close()
print('Connection closed.')


Table "sunspots" has been created without specifying data types.
Inserted 7 records into the table.
Deleted 1 records from the table where sunactivity is greater than 20.
Data where the year is less than 1732:
(1700, 5)
(1701, 11)
(1702, 16)
(1703, 18)
(1704, 11)
(1731, 11)
Connection closed.


In [15]:
# Solutions to Task 3

from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker

# Define the superclass using SQLAlchemy's declarative base
Base = declarative_base()

# Define the Station class
class Station(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    location = Column(String)

# Define the Sensor class
class Sensor(Base):
    __tablename__ = 'sensors'
    
    id = Column(Integer, primary_key=True)
    type = Column(String)
    location = Column(String)
    reading = Column(Float)

# Create an engine that stores data in the in-memory SQLite database
engine = create_engine('sqlite:///:memory:')

# Create all tables in the engine. This is equivalent to "Create Table" statements in raw SQL.
Base.metadata.create_all(engine)

# Create a DBSession, which is the staging zone for all the objects loaded into the database session object.
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Create two stations and commit the session
station1 = Station(name='Station 1', location='Location 1')
station2 = Station(name='Station 2', location='Location 2')
session.add(station1)
session.add(station2)
session.commit()

# Insert a sensor record
sensor = Sensor(type='Temperature', location='Room 1', reading=23.5)
session.add(sensor)
session.commit()

# Output message
'Two stations and one sensor record have been inserted.'


'Two stations and one sensor record have been inserted.'

In [16]:
# Solutions to Task 4
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

# Define the superclass using SQLAlchemy's declarative base
Base = declarative_base()

# Define the Station class
class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    location = Column(String)
    # Establish a relationship with Sensor
    sensors = relationship("Sensor", back_populates="station")

# Define the Sensor class
class Sensor(Base):
    __tablename__ = 'sensors'
    id = Column(Integer, primary_key=True)
    type = Column(String)
    location = Column(String)
    reading = Column(Float)
    # Define a foreign key that points to the stations table
    station_id = Column(Integer, ForeignKey('stations.id'))
    # Establish a relationship with Station
    station = relationship("Station", back_populates="sensors")

# Create an engine from a URI
engine = create_engine('sqlite:///example.db')

# Create all tables in the engine. This is equivalent to "Create Table" statements in raw SQL.
Base.metadata.create_all(engine)

# Create a DBSession object
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Add a station record to ensure the table is not empty
station = Station(name='Weather Station', location='Central Park')
session.add(station)
session.commit()

# Query the first row in the station table
first_station = session.query(Station).first()
if first_station:
    print(f'First station: {first_station.name} located at {first_station.location}')
else:
    print('No stations found.')

# Query all the stations
all_stations = session.query(Station).all()
print('All stations:')
for station in all_stations:
    print(f'{station.name} located at {station.location}')

# Add a sensor record to ensure the table is not empty
sensor = Sensor(type='Temperature', location='Central Park', reading=23.5, station_id=first_station.id)
session.add(sensor)
session.commit()

# Query all the sensors
all_sensors = session.query(Sensor).all()
print('All sensors:')
for sensor in all_sensors:
    print(f'{sensor.type} sensor located at {sensor.location} with reading {sensor.reading}')

# Query the station table with pandas read_sql()
station_df = pd.read_sql('stations', engine)
print('Station table queried with pandas read_sql():')
print(station_df)



First station: Weather Station located at Central Park
All stations:
Weather Station located at Central Park
Weather Station located at Central Park
All sensors:
Temperature sensor located at Central Park with reading 23.5
Temperature sensor located at Central Park with reading 23.5
Station table queried with pandas read_sql():
   id             name      location
0   1  Weather Station  Central Park
1   2  Weather Station  Central Park
